In [2]:
import cv2
import mediapipe as mp
import math
import numpy as np
import torch
import cv2
import mediapipe as mp
import math
import numpy as np
from ultralytics import YOLO

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    # Convert points to numpy arrays if they are MediaPipe landmarks
    a = np.array([a.x, a.y]) if hasattr(a, 'x') else np.array(a)
    b = np.array([b.x, b.y]) if hasattr(b, 'x') else np.array(b)
    c = np.array([c.x, c.y]) if hasattr(c, 'x') else np.array(c)
    
    # Calculate vectors
    ab = a - b
    cb = c - b
    
    # Calculate dot product and magnitudes
    dot_product = np.dot(ab, cb)
    magnitude_ab = np.linalg.norm(ab)
    magnitude_cb = np.linalg.norm(cb)
    
    # Calculate angle in radians and convert to degrees
    angle_radians = np.arccos(dot_product / (magnitude_ab * magnitude_cb))
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

# Function to calculate tilt (angle of a line relative to the horizontal axis)
def calculate_tilt(a, b):
    # Convert points to numpy arrays if they are MediaPipe landmarks
    a = np.array([a.x, a.y]) if hasattr(a, 'x') else np.array(a)
    b = np.array([b.x, b.y]) if hasattr(b, 'x') else np.array(b)
    
    dx = b[0] - a[0]
    dy = b[1] - a[1]
    tilt_radians = math.atan2(dy, dx)
    tilt_degrees = math.degrees(tilt_radians)
    return tilt_degrees

# Function to calculate speed (pixels per frame)
def calculate_speed(prev_point, curr_point, frame_rate):
    dx = curr_point.x - prev_point.x
    dy = curr_point.y - prev_point.y
    distance = math.sqrt(dx**2 + dy**2)
    speed = distance * frame_rate  # Speed in pixels per second
    return speed

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
        min_detection_confidence=0.1,
        min_tracking_confidence=0.1,
        model_complexity=1)

# Initialize YOLO model for ball detection
yolo_model_racquet = YOLO("D:\\DDSA\\YOLO_Models\\yolo11l_best.pt")  # Replace with a model trained on tennis data if available
yolo_model_racquet.to('cpu')

yolo_model_ball = YOLO("D:\\DDSA\\Serve Pose Detection\\ball_detection_model.pt")  # Replace with a model trained on tennis data if available
yolo_model_ball.to('cpu')

# Path to the input video file
file_name = "JD1"
input_video_path = 'D:\\DDSA\\Basketball player detection\\' + file_name + '.mp4' # Replace with your video file path

# Open the video file
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))


# Reference object properties
reference_object_height_px = 100  # Height of the reference object in pixels
reference_object_height_cm = 150  # Height of the reference object in centimeters (e.g., a person's height)

# Calculate the conversion factor (cm/pixel)
conversion_factor = reference_object_height_cm / reference_object_height_px

# Initialize variables for speed calculation
prev_wrist_position = None

# Initialize variables for ball release and racquet contact
ball_release_time = None
racquet_contact_time = None
service_motion_time = None

# Initialize variables for ball height
current_ball_height_cm = 0
max_ball_height_cm = 0

# Initialize variables for ball height
ball_ankle_height_cm = 0
max_ball_height_cm = 0

# Initialize variables for ball tracking
ball_position = None
prev_ball_position = None

# Custom drawing specifications - brighter colors for better visibility
landmark_drawing_spec = mp_drawing.DrawingSpec(
        color=(0, 255, 255),  # Bright yellow for landmarks
        thickness=3,
        circle_radius=3
    )
connection_drawing_spec = mp_drawing.DrawingSpec(
        color=(255, 0, 255),  # Bright magenta for connections
        thickness=3,
        circle_radius=3
    )
# Path to the input video file
file_name_list = ['5']

# Loop through listed file names
for file in file_name_list:
    input_video_path = 'D:\\DDSA\\Serve Pose Detection\\Test Set\\Sent by shane\\' + file + '.mp4'
    output_video_path = 'D:\\DDSA\\Serve Pose Detection\\Test Set\\Sent by shane\\Output\\' + file + '_output.mp4'

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image and detect poses
        results_pose = pose.process(image_rgb)

        # Detect the racquet using YOLO
        try:
            results_yolo_racquet = yolo_model_racquet(frame, conf=0.1)
        except Exception as e:
            print(f"YOLO Error: {e}")
            results_yolo_racquet = None
        
            
        # Extract racquet detections
        racquet_position = None
        if results_yolo_racquet:
            for result in results_yolo_racquet:
                boxes = result.boxes  # Bounding boxes
                print(result.boxes.cls)
                for box in boxes:
                    if box.cls == 0:  
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # Bounding box coordinates
                        racquet_position = ((x1 + x2) / 2, (y1 + y2) / 2)  # Ball center
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                        cv2.circle(frame, (int(racquet_position[0]), int(racquet_position[1])), 5, (0, 255, 0), -1)

        # Detect the ball using YOLO
        try:
            results_yolo_ball = yolo_model_ball(frame, conf=0.1)
        except Exception as e:
            print(f"YOLO Error: {e}")
            results_yolo_ball = None

        
        # Extract ball detections
        ball_position = None
        if results_yolo_ball:
            for result in results_yolo_ball:
                boxes = result.boxes  # Bounding boxes
                print(result.boxes.cls)
                for box in boxes:
                    if box.cls == 0:  
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # Bounding box coordinates
                        ball_position = ((x1 + x2) / 2, (y1 + y2) / 2)  # Ball center
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                        cv2.circle(frame, (int(ball_position[0]), int(ball_position[1])), 5, (0, 255, 0), -1)

        

        # Process pose estimation results
        if results_pose.pose_landmarks:
            # Draw the pose annotation on the frame
            mp_drawing.draw_landmarks(
                        frame, 
                        results_pose.pose_landmarks, 
                        mp_pose.POSE_CONNECTIONS,
                        landmark_drawing_spec,
                        connection_drawing_spec)

            # Get keypoints
            landmarks = results_pose.pose_landmarks.landmark
            shoulder_left = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
            shoulder_right = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            elbow_left = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
            elbow_right = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
            wrist_left = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
            wrist_right = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
            hip_left = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
            hip_right = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
            knee_left = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
            knee_right = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
            ankle_left = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
            ankle_right = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]

            # Calculate vertical distance between ball and ankle (if ball is detected)
            if ball_position is not None:
                ankle_position = (int(ankle_right.x * frame_width), int(ankle_right.y * frame_height))
                ball_ankle_height_px = ankle_position[0] - ball_position[0]  # Positive if ball is above the ankle
                ball_ankle_height_cm = ball_ankle_height_px * conversion_factor

                # Update maximum ball height
                if ball_ankle_height_cm > max_ball_height_cm:
                    max_ball_height_cm = ball_ankle_height_cm

            # Calculate upper body tilt (shoulder line relative to horizontal)
            upper_body_tilt = calculate_tilt(shoulder_left, shoulder_right)

            # Calculate arm angle (shoulder, elbow, wrist)
            arm_angle = calculate_angle(shoulder_right, elbow_right, wrist_right)

            # Calculate elbow flexion (shoulder, elbow, wrist)
            elbow_flexion = calculate_angle(shoulder_right, elbow_right, wrist_right)

            # Calculate wrist flexion (elbow, wrist, [wrist + offset])
            wrist_flexion = calculate_angle(elbow_right, wrist_right, (wrist_right.x + 0.1, wrist_right.y))  # Offset for wrist direction

            # Calculate arm speed (wrist movement between frames)
            if prev_wrist_position:
                arm_speed = calculate_speed(prev_wrist_position, wrist_right, fps)
            prev_wrist_position = wrist_right

            # Calculate left and right knee flexion (hip, knee, ankle)
            left_knee_flexion = calculate_angle(hip_left, knee_left, ankle_left)
            right_knee_flexion = calculate_angle(hip_right, knee_right, ankle_right)

            # Calculate current ball height (wrist position in cm)
            current_ball_height_cm = (ankle_left.y + frame_height) * conversion_factor

            # Update maximum ball height
            if current_ball_height_cm > max_ball_height_cm:
                max_ball_height_cm = current_ball_height_cm

            # Detect ball release (when wrist reaches its highest point)
            if prev_wrist_position and wrist_right.y > prev_wrist_position.y:
                ball_release_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds

            # Detect racquet contact (when wrist starts moving downward)
            if prev_wrist_position and wrist_right.y < prev_wrist_position.y:
                racquet_contact_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds

            # Calculate service motion time
            if ball_release_time is not None and racquet_contact_time is not None:
                service_motion_time = racquet_contact_time - ball_release_time

            # Calculate the time difference between ball release and racquet contact
            if ball_release_time is not None and racquet_contact_time is not None:
                time_difference = racquet_contact_time - ball_release_time
                cv2.putText(frame, f"Ball Release to Racquet Contact: {time_difference:.3f} s", (10, 280), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Calculate ball projection angle (shoulder, wrist, [wrist + offset])
            ball_projection_angle = calculate_angle(shoulder_right, wrist_right, (wrist_right.x + 0.1, wrist_right.y))

            # Calculate lower body tilt (hip line relative to horizontal)
            lower_body_tilt = calculate_tilt(hip_left, hip_right)

            # Calculate trunk tilt (shoulder midpoint, hip midpoint)
            shoulder_midpoint = ((shoulder_left.x + shoulder_right.x) / 2, (shoulder_left.y + shoulder_right.y) / 2)
            hip_midpoint = ((hip_left.x + hip_right.x) / 2, (hip_left.y + hip_right.y) / 2)
            trunk_tilt = calculate_tilt(shoulder_midpoint, hip_midpoint)

            # Display results on the frame
            cv2.putText(frame, f"Upper Body Tilt: {upper_body_tilt:.2f} deg", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Arm Angle: {arm_angle:.2f} deg", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Elbow Flexion: {elbow_flexion:.2f} deg", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Wrist Flexion: {wrist_flexion:.2f} deg", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #cv2.putText(frame, f"Arm Speed: {arm_speed:.2f} px/s", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Left Knee Flexion: {left_knee_flexion:.2f} deg", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Right Knee Flexion: {right_knee_flexion:.2f} deg", (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Ball Height: {current_ball_height_cm:.2f} cm", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Max Ball Height: {max_ball_height_cm:.2f} cm", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            if service_motion_time is not None:
                cv2.putText(frame, f"Service Motion Time: {service_motion_time:.3f} s", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Ball Projection Angle: {ball_projection_angle:.2f} deg", (10, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Lower Body Tilt: {lower_body_tilt:.2f} deg", (10, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f"Trunk Tilt: {trunk_tilt:.2f} deg", (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            cv2.putText(frame, f"Ball Height Above Ankle: {ball_ankle_height_cm:.2f} cm", (10, 320), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #cv2.putText(frame, f"Max Ball Height: {max_ball_height_cm:.2f} cm", (10, 340), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Draw a line between the ankle and the ball for visualization
            # cv2.line(frame, (int(ball_position[0]), int(ball_position[1])), ankle_position, (255, 0, 0), 2)
        # Write the frame to the output video file
        out.write(frame)

# Release resources
cap.release()
out.release()

print(f"Processed video saved to: {output_video_path}")
print(f"Maximum Ball Height: {max_ball_height_cm:.2f} cm")
if service_motion_time is not None:
    print(f"Service Motion Time: {service_motion_time:.3f} s")
if ball_release_time is not None and racquet_contact_time is not None:
    time_difference = racquet_contact_time - ball_release_time
    print(f"Time difference between ball release and racquet contact: {time_difference:.3f} seconds")


0: 480x640 1 racquet , 614.5ms
Speed: 7.4ms preprocess, 614.5ms inference, 10.4ms postprocess per image at shape (1, 3, 480, 640)
tensor([0.])

0: 480x640 (no detections), 973.6ms
Speed: 2.2ms preprocess, 973.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
tensor([])

0: 480x640 1 racquet , 510.7ms
Speed: 2.4ms preprocess, 510.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
tensor([0.])

0: 480x640 (no detections), 994.0ms
Speed: 2.0ms preprocess, 994.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
tensor([])

0: 480x640 1 racquet , 506.7ms
Speed: 3.4ms preprocess, 506.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
tensor([0.])

0: 480x640 (no detections), 973.3ms
Speed: 1.0ms preprocess, 973.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
tensor([])

0: 480x640 (no detections), 526.8ms
Speed: 3.1ms preprocess, 526.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 64

KeyboardInterrupt: 